# Ekster

Ekster is an AMUSE code designed to simulate star-forming regions, without needing very high mass resolution for the gas (approximately 0.1-1 MSun per gas particle).
It combines hydrodynamics (using SPH) with gravity and (optional) stellar evolution, by default using the codes Phantom [https://phantomsph.bitbucket.io], PeTar [https://github.com/lwang-astro/PeTar] and SeBa [https://github.com/amusecode/SeBa].

The original use for Ekster was to re-simulate a part of a gas-only galaxy simulation to the point where star clusters would form, and evolve these young clusters for a short time [https://ui.adsabs.harvard.edu/abs/2022MNRAS.509.6155R/abstract].

Similar to other star-forming region simulation codes, Ekster creates sink particles from gas that is collapsing to form stars. What is different is that in Ekster these sinks are then converted into individual stars, which represent the stars born in proto-clusters. An initial mass function and initial spread of the stars is implied in this step.

The gas and the stars are then integrated together in their respective codes via Bridge, optionally with an external (time-dependent) tidal field applied to both [https://ui.adsabs.harvard.edu/abs/2013MNRAS.436.3695R/abstract].

Stellar feedback (ionisation and stellar winds) is under development.

## Installing Ekster and its prerequisites

You may skip this step if you have already done this.

If you are using the Developer installation of AMUSE, you will need to checkout the git repositories of Ekster and MASC, and copy (or symlink) the files to src/amuse/ext in the AMUSE repository.

In [ ]:
!pip install mpi4py amuse-framework amuse-phantom amuse-fi amuse-petar amuse-seba
!pip install amuse-ekster amuse-masc

## Using Ekster

Currently, the interface differs from that of a 'regular' community code, since it was developed as a script initially. Future versions will have an interface in line with that of AMUSE community codes, but for now I explain how to use the current interface below.

In [ ]:
import time as clock
import numpy as np
# Please change this seed to something else at your liking
np.random.seed(20221012)

In [ ]:
# Import standard AMUSE dependencies
from amuse.units import units, nbody_system
from amuse.ic.molecular_cloud import new_molecular_cloud
from amuse.io import write_set_to_file

In [ ]:
# Import main modules for Ekster...
from amuse.ext.ekster import ClusterInPotential, Settings
from amuse.ext.ekster import ekster_settings
# ...and also some helper functions
from amuse.ext.ekster.plotting_class import (
    u_to_temperature, temperature_to_u,
    gas_mean_molecular_weight,
)

In [ ]:
# Settings are stored as properties of the Settings class, which is initialised with safe defaults:
settings = Settings()

print(
    f"For hydrodynamics, we use {settings.sph_code}. "
    f"Stars are integrated using {settings.star_code}, "
    f"and stellar evolution is handled by {settings.evo_code}."
)
print(
    "Feedback-related settings: \n"
    f"Phantom cooling (1 = on, 0 = off): {settings.icooling}\n"
    f"Phantom EOS (1 = isothermal, 2 = adiabatic): {settings.ieos}\n"
    f"Stellar masses evolve: {settings.evo_stars_lose_mass}\n"
    f"Stellar winds: {settings.wind_enabled}\n"
    f"Stellar wind type: {settings.wind_type}\n"
    f"Radiative feedback (under development): {settings.feedback_enabled}\n"
)
# print(dir(ekster_settings))

In [ ]:
# If you want to run a simulation with an adiabatic EOS and cooling, run this cell.
settings.icooling = 1
settings.ieos = 2

## Creating a star-forming region
Once gas reaches a critical density, Ekster will check if the region is indeed collapsing.
If this is the case, it will create a sink particle, which accretes all gas within a specified radius.
The sink will then act as a star-forming region, creating stars within its radius (initially set to the same value as the accretion radius, but then shrinking to keep a constant density).

In [ ]:
print(
    f"Density threshold for forming sinks: {settings.density_threshold}\n"
    f"Accretion radius for sinks: {settings.minimum_sink_radius}\n"
    # this setting's name is a legacy from the development phase
)

## Star formation method
Ekster has two methods for forming stars from sinks: 'single' and 'grouped'. The 'single' method converts every sink into a reservoir for star formation, forming stars until the remaining mass is not sufficient for a new star.
The 'grouped' method uses a set of nearby sinks of similar age as a group reservoir, combining the mass of these sinks to form new stars [https://ui.adsabs.harvard.edu/abs/2022MNRAS.510.2657L/abstract].

This method is primarily useful when the mass of an individual sink would be lower than required to probe the IMF (which would result in sinks with remaining mass). If sinks start with ~250 MSun, the 'single' method is generally preferred. You can check if the 'grouped' method might be beneficial by comparing the mass left in sinks to the total mass in stars - if this is a high fraction, you might want to look into using the 'grouped' method.

Currently Ekster assumes 100% of the mass accreted onto a sink is converted into stars, a future version may allow setting this efficiency rate (e.g. converting remaining mass back into gas, to be expelled).

In [ ]:
print(settings.star_formation_method)

## Generate a molecular cloud to start with

You can play around with the parameters, but in order to form stars the cloud needs to collapse (following the Jeans criterion). Stars will start to form once the density of the gas reaches 10<sup>-18</sup> g/cm<sup>3</sup>.

In [ ]:
gas_density = 1e-20 | units.g * units.cm**-3
# Increasing 'volume_scaling' will give a larger cloud with similar properties - but it will run much slower!
volume_scaling = 2
number_of_gas_particles = volume_scaling**3 * 1000
total_mass_of_gas_particles = number_of_gas_particles | units.MSun
volume = total_mass_of_gas_particles / gas_density
cloud_radius = volume_scaling * (volume / (units.pi * 4/3))**(1/3)

converter_gas = nbody_system.nbody_to_si(total_mass_of_gas_particles, cloud_radius)
gas = new_molecular_cloud(targetN=number_of_gas_particles, convert_nbody=converter_gas)
gas.collection_attributes.timestamp = 0 | units.yr

# The temperature of the gas is ignored in isothermal simulations (but still needs to be set), 
# in other simulations it will quickly adjust to an equilibrium temperature
gas.u = temperature_to_u(30 | units.K)

In [ ]:
# We want to disable redirection, since it gives a lot of extra output
# - which is great for debugging, but not ideal in a notebook...

settings.code_redirection = "null"
simulation = ClusterInPotential(
    gas=gas,
    settings=settings,
)
simulation.sync_from_gas_code()

In [ ]:
# Let's define a way to backup our simulation - it can take a while, 
# so we want to store any important steps on the way!

def write_backup(model, step=0, settings=settings, run_prefix="MyEksterSimulation-"):
    if not model.gas_particles.is_empty():
        settings.filename_gas = f"{run_prefix}gas-{step:04d}.amuse"
    if not model.star_particles.is_empty():
        settings.filename_stars = f"{run_prefix}stars-{step:04d}.amuse"
    if not model.sink_particles.is_empty():
        settings.filename_sinks = f"{run_prefix}sinks-{step:04d}.amuse"
    settings.filename_random = \
        f"{run_prefix}randomstate-{step:04d}.pkl"
    settings.step = step
    settings.model_time = model.model_time
    # Write the current settings to a file.
    ekster_settings.write_config(
        settings, f"{run_prefix}resume-{step:04d}.ini", "resume"
    )
    # Write the particle sets (gas, sinks, stars).
    if not model.gas_particles.is_empty():
        model.gas_particles.collection_attributes.timestamp = (
            model.gas_code.model_time
        )
        write_set_to_file(
            model.gas_particles,
            settings.filename_gas,
            append_to_file=False,
            compression="gzip",
            close_file=True,
        )
    if not model.sink_particles.is_empty():
        model.sink_particles.collection_attributes.timestamp = (
            model.sink_code.model_time
        )
        write_set_to_file(
            model.sink_particles,
            settings.filename_sinks,
            append_to_file=False,
            compression="gzip",
            close_file=True,
        )
    if not model.star_particles.is_empty():
        model.star_particles.collection_attributes.timestamp = (
            model.star_code.model_time
        )
        write_set_to_file(
            model.star_particles,
            settings.filename_stars,
            append_to_file=False,
            compression="gzip",
            close_file=True,
        )

In [ ]:
# It may be good to save a snapshot of our initial conditions at this point, at least to test if it works.
# Make sure to give it an unique name - files won't be overwritten!
model_name = "MyEksterSimulationAdiabatic"

write_backup(simulation, step=0, settings=settings, run_prefix=model_name)

In [ ]:
# Let's now try to evolve for one timestep
time_start = clock.time() | units.s
simulation.evolve_model(simulation.timestep)
step = 1
time_end = clock.time() | units.s
# and save a backup
write_backup(simulation, step=step, settings=settings, run_prefix=model_name)

step_time = simulation.timestep/(time_end-time_start)
print(f"One step took {(time_end-time_start).value_in(units.s):.2f} seconds - that's {step_time} x reality")

In [ ]:
print(len(simulation.sink_particles))
print(len(simulation.star_particles))
print(simulation.gas_particles.density.max())

Let's now evolve the system until the first stars form.
In an isothermal simulation, the timescale for this is similar to the dynamical timescale of the system.

In [ ]:
simulation_time_dynamical = simulation.gas_particles.dynamical_timescale()
print(f"One dynamical time: {simulation_time_dynamical}\n")
print(f"Approximately {int(simulation_time_dynamical/simulation.timestep)} steps")
print(f"This will take approximately {(simulation_time_dynamical/step_time).value_in(units.minute):.2f} minutes to run")

If this takes too long, you might want to create a smaller system - or a denser one, so that it collapses sooner.

In [ ]:
time_start = clock.time() | units.s
simulation_time_start = simulation.model_time
simulation_step_start = step
while simulation.sink_particles.is_empty():
    simulation.evolve_model(simulation.model_time + simulation.timestep)
    step += 1
time_end = clock.time() | units.s
simulation_time_end = simulation.model_time
simulation_step_end = step

time_firststars = simulation.model_time
simulation_duration = time_end - time_start
steps = simulation_step_end - simulation_step_start
step_time = (time_end-time_start) / steps

In [ ]:
print(f"The first stars form at {time_firststars} ({step} steps)")
print(f"This took {simulation_duration.value_in(units.minute):.2f} minutes")
print(f"An average step now takes {step_time.value_in(units.s):.2f} seconds")

# let's save a backup again
write_backup(simulation, step=step, settings=settings, run_prefix=model_name)

In [ ]:
# We don't have any sinks or stars yet.
print(len(simulation.sink_particles))
print(len(simulation.star_particles))

In [ ]:
print(f"The stars have a total mass of {simulation.star_particles.total_mass()}")
print(f"The sinks have a total mass of {simulation.sink_particles.total_mass()}")

In [ ]:
# Let's make a plot
from amuse.ext.ekster.plotting_class import plot_hydro_and_stars

In [ ]:
fig, ax = plot_hydro_and_stars(
    simulation.model_time,
    gas=simulation.gas_particles,
    stars=simulation.star_particles,
    sinks=simulation.sink_particles,
    settings=settings,
    return_figure=True,
)

In [ ]:
# Let's now evolve until we have a second sink.
# Note that from this point, we are evolving both the SPH and gravity codes.
# The code will run slower as a result.
time_start = clock.time() | units.s
simulation_time_start = simulation.model_time
simulation_step_start = step
while len(simulation.sink_particles) < 2:
    simulation.evolve_model(simulation.model_time + simulation.timestep)
    step += 1
time_end = clock.time() | units.s
simulation_time_end = simulation.model_time
simulation_step_end = step

simulation_duration = time_end - time_start
step_time = simulation_duration / (time_end-time_start)
time_secondstars = simulation.model_time

steps = simulation_step_end - simulation_step_start
step_time = (time_end-time_start) / steps
# let's save a backup again
write_backup(simulation, step=step, settings=settings, run_prefix=model_name)

In [ ]:
print(f"The second group of stars form at {time_secondstars}")
print(f"It took {simulation_duration.value_in(units.minute):.2f} minutes")
print(f"In this phase we are taking {step_time.value_in(units.s):.2f} seconds per step")

In [ ]:
# Make sure all codes are stopped
simulation.stop()
simulation.gas_code.code.stop()

## Visualise it with Fresco!

In [ ]:
!pip install --upgrade amuse-fresco

In [ ]:
# Import Fresco and matplotlib.
from amuse.plot.fresco import make_fresco_image
import matplotlib.pyplot as plt

In [ ]:
image_width = 5 | units.pc
image_width_in_pc = image_width.value_in(units.pc)
simulation.gas_particles.radius = simulation.gas_particles.h_smooth
fresco_image, vmax = make_fresco_image(
    stars=simulation.star_particles,
    gas=simulation.gas_particles,
    converter=simulation.converter,
    image_width=image_width,
    return_vmax=True,
    extinction=True,
)
extent = [-image_width_in_pc/2, image_width_in_pc/2, -image_width_in_pc/2, image_width_in_pc/2]
plt.imshow(fresco_image, extent=extent, origin="lower")

Ekster also has support for Fresco built in - just use the same plotting strategy as before, but now pass the keyword "use_fresco" for a Fresco blend-in to the original image (0 = no Fresco, 1 = only Fresco)

In [ ]:
fig, ax = plot_hydro_and_stars(
    simulation.model_time,
    gas=simulation.gas_particles,
    stars=simulation.star_particles,
    sinks=simulation.sink_particles,
    settings=settings,
    return_figure=True,
    use_fresco=1.0,
)